# Analyzing LAPD's 'Part I' crimes database: 2010-present

### Import data tools

In [1]:
import pandas as pd
import geopandas as gpd
import pyarrow
import matplotlib
import matplotlib.pyplot as plt
import geojson
import json
import jenkspy
import numpy as np
from earthpy import clip as cl
from altair import datum
import altair as alt
import altair_latimes as lat
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000

### Read crimes CSV downloaded from LA City data portal

In [ ]:
# data exported from '00-lapd-crimes-processing.ipynb'
crimes = pd.read_feather('/Users/mhustiles/data/data/LA/crimes.feather')

In [19]:
crimes.sort_values(by="date_occurred", ascending=False).head()

,index,record_id,date_reported,date_occurred,time_occurred,division,division_name,reporting_district,part_type,crime_code,crime_code_description,modus_operandi_code,victim_age,victim_sex,victim_descent,premises_code,premises_description,weapon_code,weapon_description,status_code,status_code_description,crm_cd_1,crm_cd_2,crm_cd_3,crm_cd_4,address,cross_street,latitude,longitude,year,quarter,day,month,weekday,monthname,descent_description,hour,minute,part_category
1069429,1935273,201304012,2020-01-01,2019-12-31,2000,13,Newton,1346,1,510,Vehicle - stolen,None,0,None,None,101,Street,nan,None,IC,invest_cont,510,nan,nan,nan,1100 E VERNON AV,None,34.0060,-118.2513,2019,4,31,12,Tuesday,December,None,20,00,pt_one_property
1084135,1960636,190224014,2019-12-31,2019-12-31,0410,2,Rampart,219,1,230,"Assault with deadly weapon, aggravated assault",1266 0342 1822 0431,60,male,H,108,Parking lot,512.0,Mace/pepper spray,IC,invest_cont,230,nan,nan,nan,1100 BELLEVUE AV,None,34.0653,-118.2493,2019,4,31,12,Tuesday,December,hispanic,04,10,pt_one_violent
1052160,1905219,200604020,2020-01-01,2019-12-31,2015,6,Hollywood,643,1,310,Burglary,2037 1822 0344,37,male,O,501,Single family dwelling,nan,None,IC,invest_cont,310,nan,nan,nan,7400 HOLLYWOOD BL,None,34.1017,-118.3511,2019,4,31,12,Tuesday,December,other,20,15,pt_one_property
1053241,1907052,200104026,2019-12-31,2019-12-31,1930,1,Central,162,1,442,Shoplifting - petty theft ($950 & under),1822 0325,0,male,O,403,Drug store,nan,None,IC,invest_cont,442,nan,nan,nan,600 W 7TH ST,None,34.0473,-118.2566,2019,4,31,12,Tuesday,December,other,19,30,pt_one_property
1059279,1917696,191324772,2019-12-31,2019-12-31,0830,13,Newton,1327,1,236,Intimate partner - aggravated assault,2000 0913 0416,47,male,H,501,Single family dwelling,500.0,Unknown weapon/other weapon,AO,adult_other,236,nan,nan,nan,1500 E 23RD ST,None,34.0181,-118.2488,2019,4,31,12,Tuesday,December,hispanic,08,30,pt_one_violent


### Which crimes are most common? 

In [7]:
violations = crimes.groupby(['crime_code_description'])\
.agg('size').reset_index(name='total').sort_values(by='total', ascending=False)

In [8]:
violations.head(10)

,crime_code_description,total
9,Burglary from vehicle,162050
54,Vehicle - stolen,159722
42,Theft plain - petty ($950 & under),149561
8,Burglary,147409
2,"Assault with deadly weapon, aggravated assault",92632
39,Theft from motor vehicle - petty ($950 & under),88368
29,Robbery,83813
47,"Theft-grand ($950.01 & over)excpt,guns,fowl,li...",74299
32,Shoplifting - petty theft ($950 & under),48380
38,Theft from motor vehicle - grand ($400 and over),31605


### Places where crimes occur?

In [9]:
places = crimes.groupby(['premises_description'])\
.agg('size').reset_index(name='total').sort_values(by='total', ascending=False)

In [10]:
places.head(10)

,premises_description,total
273,Street,358571
262,Single family dwelling,146704
229,Parking lot,115353
207,"Multi-unit dwelling (apartment, duplex, etc)",85681
261,Sidewalk,61525
220,Other business,50238
64,Driveway,34295
80,Garage/carport,33409
56,Department store,26478
108,Market,18316


### Areas of the city? 

In [11]:
divisions = crimes.groupby(['division_name'])\
.agg('size').reset_index(name='total').sort_values(by='total', ascending=False)

In [12]:
divisions.head(10)

,division_name,total
0,77th Street,79948
15,Southwest,75950
12,Pacific,67790
8,N Hollywood,63241
14,Southeast,61532
9,Newton,57847
10,Northeast,55868
6,Hollywood,55114
1,Central,55076
2,Devonshire,54583


### What kinds of weapons?

In [13]:
weapons = crimes.groupby(['weapon_description'])\
.agg('size').reset_index(name='total').sort_values(by='total', ascending=False)

In [14]:
weapons.head(10)

,weapon_description,total
65,"Strong-arm (hands, fist, feet or bodily force)",105348
75,Unknown weapon/other weapon,33139
27,Hand gun,31527
58,Semi-automatic pistol,12050
32,Knife with blade 6inches or less,11801
44,Other knife,8993
77,Vehicle,6615
73,Unknown firearm,6265
50,Revolver,5203
78,Verbal threat,5121


### Which days of the week?

In [15]:
weekdays = crimes.groupby(['weekday'])\
.agg('size').reset_index(name='total').sort_values(by='total', ascending=False)

In [16]:
weekdays.head(7)

,weekday,total
0,Friday,181811
2,Saturday,170304
6,Wednesday,165761
4,Thursday,165681
1,Monday,163879
5,Tuesday,163442
3,Sunday,159098


---

### How does this year compare with the past?